#### LINK TO DOWNLOAD NEO4J

http://neo4j.com/artifact.php?name=neo4j-community-2.3.1-unix.tar.gz
 
#### AWS SAMPLE CODE

https://alestic.com/2013/11/aws-cli-query/

# Attempt to automate configuration

In [ ]:
%%bash
#'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# W205 Final Project: Million Song Dataset (MSD)

# Requirements: W205 AMI with Hadoop and Spark
#               aws cli installed and configured (run the following:
#  $aws configure
#  AWS Access Key ID [None]: <access key>
#  AWS Secret Access Key [None]: <secret access key>
#  Default region name [None]: us-east-1
#  Default output format [None]: json

# This configurations scripts is run from within the EC2 instance.
# It assumes that the instance DOES NOT have any volume attached and that the mount
# point /data is available
 
# Python Libraries: py2neo,
#'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

# AS ROOT
# === Installations ===
sudo yum install jq
pip install awscli
 
# === Install ec2-metadata tool to get information about this instance ===
wget http://s3.amazonaws.com/ec2metadata/ec2-metadata
chmod a+x ec2-metadata
mv ec2-metadata /usr/bin
 
# ============================================================================================================================
#'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# AWS Setup - Attache 2 volumes to this instance
# Main Volume:
# MSD VolumeL  is where the Million Song Dataset (MSD)
#'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

# === Save instance info in environment variables ===
# Get instance id
INSTANCE_ID=$(ec2-metadata -i | cut -d:  -f2| cut -d' ' -f2)
# Get instance public hostname
INSTANCE_PDNS=$(ec2-metadata -p | cut -d:  -f2| cut -d' ' -f2)
# Get instance availability zone
INSTANCE_ZONE=$(ec2-metadata -z | cut -d:  -f2| cut -d' ' -f2)


export INSTANCE_ID
export INSTANCE_PDNS
export INSTANCE_ZONE

echo 'export INSTANCE_ID='$INSTANCE_ID >> ~/.bashrc
echo 'export INSTANCE_PDNS='$INSTANCE_PDNS >> ~/.bashrc
echo 'export INSTANCE_ZONE='$INSTANCE_ZONE >> ~/.bashrc

source ~/.bashrc

# === Create and Attache Volumes ===
mkdir aws-info
 
### Create project main working volume
aws ec2 create-volume --size 100 --availability-zone $INSTANCE_ZONE --volume-type gp2 > aws-info/main-volume.json
wait
MAIN_VOL_ID = jq '.VolumeId' aws-info/main-volume.json
 
aws ec2 attach-volume --volume-id $MAIN_VOL_ID --instance-id $INSTANCE_ID --device /dev/xvdf
wait
mkdir data
sudo mkfs -t ext4 /dev/xvdf
sudo mount /dev/xvdf /data
 
### Create volume from AWS snapshot of Million Song Dataset (full dataset)
aws ec2 create-volume --availability-zone $INSTANCE_ZONE --snapshot-id snap-5178cf30 --volume-type gp2 > aws-info/msd-volume.json
wait
MSD_VOL_ID = jq '.VolumeId' aws-info/msd-volume.json
 
aws ec2 attach-volume --volume-id $MSD_VOL_ID --instance-id $INSTANCE_ID --device /dev/xvdg
wait
mkdir msong_data
sudo mount /dev/xvdg /msong_data
 
# ============================================================================================================================
# === Install Neo4j in main directory ===
 
cd /data
wget http://neo4j.com/artifact.php?name=neo4j-community-2.3.1-unix.tar.gz
tar -xf neo4j-community-2.3.1-unix.tar.gz
export NEO4J_HOME="/data/neo4j"